In [ ]:
pip install ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install ipynb

In [ ]:
from pandas_datareader import data as pdr
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pandas as pd
from datetime import date
import numpy as np
import math
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
from datetime import date
import glob, os
from joblib import dump, load
from ipynb.fs.full.Data_Preparation import getFilenames
from ipynb.fs.full.Data_Preparation import getProcessedData
from operator import concat
import random
random.seed(42)

In [ ]:
path = os.getcwd()

In [ ]:
# Define the list of currency tickers used in the analysis
fx_tic = ['USDEUR',
            'USDJPY',
            'USDGBP',
            'USDCHF',
            'USDNZD',
            'USDCAD',
            'USDSEK',
            'USDDKK',
            'USDNOK',
            'EURJPY',
            'EURGBP',
            'EURCHF',
            'EURNZD',
            'EURCAD',
            'EURSEK',
            'EURDKK',
            'EURNOK']

# Create a dictionary that maps each currency ticker to an index
fx_dic = {'USDEUR': 0,
 'USDJPY': 1,
 'USDGBP': 2,
 'USDCHF': 3,
 'USDNZD': 4,
 'USDCAD': 5,
 'USDSEK': 6,
 'USDDKK': 7,
 'USDNOK': 8,
 'EURJPY': 9,
 'EURGBP': 10,
 'EURCHF': 11,
 'EURNZD': 12,
 'EURCAD': 13,
 'EURSEK': 14,
 'EURDKK': 15,
 'EURNOK': 16}

In [ ]:
# Create a dictionary that defines the top 20 features for each currency ticker. These features are selected by SFFS
feat_20={
    'USDEUR': [0, 6, 11, 12, 16, 22, 23, 24, 25, 28, 30, 31, 33, 38, 44, 45, 48, 49, 54, 59],
    'USDJPY': [0, 13, 14, 15, 16, 18, 19, 20, 21, 22, 26, 28, 31, 33, 35, 38, 41, 47, 51, 53],
    'USDGBP': [1, 2, 5, 13, 16, 18, 23, 24, 25, 31, 35, 36, 38, 41, 42, 46, 48, 51, 52, 54],
    'USDCHF': [2, 5, 7, 8, 10, 11, 17, 25, 27, 35, 41, 43, 47, 48, 51, 52, 53, 54, 55, 57],
    'USDNZD': [5, 6, 8, 10, 13, 14, 17, 20, 21, 23, 24, 25, 33, 37, 40, 43, 51, 52, 53, 57],
    'USDCAD': [5, 8, 10, 11, 17, 18, 20, 23, 24, 25, 31, 33, 35, 40, 42, 52, 54, 55, 56, 57],
    'USDSEK': [7, 8, 10, 11, 17, 18, 21, 23, 24, 25, 26, 27, 31, 33, 35, 40, 42, 43, 53, 55],
    'USDDKK': [1, 3, 7, 13, 14, 17, 22, 23, 26, 27, 28, 29, 30, 31, 35, 44, 48, 51, 52, 53],
    'USDNOK': [0, 2, 4, 5, 7, 15, 17, 18, 20, 22, 23, 26, 27, 33, 34, 37, 47, 49, 51, 57],
    'EURJPY': [0, 1, 4, 9, 13, 20, 21, 23, 24, 25, 26, 29, 31, 41, 44, 47, 51, 53, 54, 56],
    'EURGBP': [4, 5, 7, 8, 13, 17, 18, 20, 21, 23, 24, 25, 27, 41, 42, 47, 51, 52, 54, 55],
    'EURCHF': [4, 11, 12, 14, 17, 18, 19, 26, 29, 32, 33, 34, 39, 40, 45, 47, 49, 50, 51, 52],
    'EURNZD': [8, 9, 10, 11, 16, 18, 21, 23, 26, 28, 31, 35, 36, 38, 46, 50, 51, 52, 53, 55],
    'EURCAD': [5, 8, 10, 11, 18, 20, 21, 23, 24, 25, 26, 28, 32, 33, 41, 42, 43, 46, 48, 58],
    'EURSEK': [2, 4, 8, 10, 11, 13, 15, 20, 22, 23, 33, 39, 40, 41, 44, 48, 51, 54, 55, 59],
    'EURDKK': [1, 6, 8, 10, 17, 21, 26, 30, 31, 35, 37, 43, 44, 45, 47, 48, 50, 51, 55, 58],
    'EURNOK': [2, 8, 10, 11, 13, 17, 18, 21, 23, 24, 25, 40, 41, 42, 43, 47, 53, 56, 57, 58]
}

# 1. SVM

In [ ]:
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Non-linear SVM Model
def svmnon_cap(stock_name, upload = False):
    # Data split
    X_train, X_val1, X_val2, X_test, y_train, y_val1, y_val2, y_test = getProcessedData(getFilenames(fx_tic)[fx_dic[stock_name]])
    # For each market, we only use top 20 features selected by SFFS.
    X_train = X_train.iloc[:,feat_20[stock_name]]
    X_val1 = X_val1.iloc[:,feat_20[stock_name]]
    X_val2 = X_val2.iloc[:,feat_20[stock_name]]
    X_test = X_test.iloc[:,feat_20[stock_name]]

    # Check if the model should be loaded directly
    if upload == True:
        # For ensure that the model does not change when it is used again and is easy to reproduce.
        svm = load(path + '/model_weights/' + stock_name +'svmnon.h5')
        y_pred = svm.predict(X_val2)
        # Calculating the accuracy score
        test_score = svm.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
    
    # Otherwise, train the model from scratch
    elif upload == False:
        best_score = 0

        # Grid search for best parameters
        for gamma in [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]:
            for C in [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]:
                svm = SVC(gamma=gamma, C=C,kernel = 'rbf')
                svm.fit(X_train, y_train)
                score = svm.score(X_val1, y_val1)
                if score > best_score:
                    best_score = score
                    best_parameters = {'C': C, 'gamma': gamma}
        C1 = best_parameters['C']
        gamma1 = best_parameters['gamma']
        # Training the SVM model with best parameters
        svm = SVC(gamma=gamma1, C=C1,kernel = 'rbf')
        svm.fit(X_train, y_train)

        # Save as a h5 format file
        path1 = path + '/model_weights/' + stock_name +'svmnon.h5'
        dump(svm, path1)

        y_pred = svm.predict(X_val2)
        # Calculating the accuracy score
        test_score = svm.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
    else:
        print('argument upload should be True or False')

    return test_score, balanced_score1

# 2. LR

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Logistic Regression Model
def lr_cap(stock_name, upload = False):
    # Data split
    X_train, X_val1, X_val2, X_test, y_train, y_val1, y_val2, y_test = getProcessedData(getFilenames(fx_tic)[fx_dic[stock_name]])
    # For each market, we only use top 20 features selected by SFFS.
    X_train = X_train.iloc[:,feat_20[stock_name]]
    X_val1 = X_val1.iloc[:,feat_20[stock_name]]
    X_val2 = X_val2.iloc[:,feat_20[stock_name]]
    X_test = X_test.iloc[:,feat_20[stock_name]]

    # Check if the model should be loaded directly
    if upload == True:
        # For ensure that the model does not change when it is used again and is easy to reproduce.
        lg1 = load(path + '/model_weights/' + stock_name +'lr.h5')
        # Calculating the accuracy score
        y_pred = lg1.predict(X_val2)
        # Calculating the accuracy score
        test_score = lg1.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
    
    # Otherwise, train the model from scratch
    elif upload == False:
        lg = LogisticRegression()
        best_score = 0

        for C in [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000]:
            lg = LogisticRegression(penalty='l1', solver='saga', C=C ,max_iter=8000)
            lg.fit(X_train, y_train)
            score = lg.score(X_val1, y_val1)
            if score > best_score:
                best_score = score
                best_parameters = {'C': C}
        C1 = best_parameters['C']
        lg1 = LogisticRegression(penalty='l1', solver='saga', C=C1 ,max_iter=8000)
        lg1.fit(X_train, y_train)

        path1 = path + '/model_weights/' + stock_name +'lr.h5'
        dump(lg1, path1)

        y_pred = lg1.predict(X_val2)
        # Calculating the accuracy score
        test_score = lg1.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
    else:
        print('argument upload should be True or False')

    return test_score, balanced_score1

# 3.LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# Linear Discriminant Analysis Model
def lda_cap(stock_name, upload = False):
    # Data split
    X_train, X_val1, X_val2, X_test, y_train, y_val1, y_val2, y_test = getProcessedData(getFilenames(fx_tic)[fx_dic[stock_name]])
    # For each market, we only use top 20 features selected by SFFS.
    X_train = X_train.iloc[:,feat_20[stock_name]]
    X_val1 = X_val1.iloc[:,feat_20[stock_name]]
    X_val2 = X_val2.iloc[:,feat_20[stock_name]]
    X_test = X_test.iloc[:,feat_20[stock_name]]

    # Check if the model should be loaded directly
    if upload == True:
        # For ensure that the model does not change when it is used again and is easy to reproduce.
        lda = load(path + '/model_weights/' + stock_name +'lda.h5')
        y_pred = lda.predict(X_val2)
        # Calculating the accuracy score
        test_score = lda.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
        
    # Otherwise, train the model from scratch
    elif upload == False:
        lda = LinearDiscriminantAnalysis()
        X_train1 = pd.concat([X_train, X_val1],axis=0)
        y_train1 = pd.concat([y_train, y_val1],axis=0)
        lda.fit(X_train1, y_train1)

        path1 = path + '/model_weights/' + stock_name +'lda.h5'
        dump(lda, path1)

        y_pred = lda.predict(X_val2)
        # Calculating the accuracy score
        test_score = lda.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
    else:
        print('argument upload should be True or False')

    return test_score, balanced_score1


# 4.QDA

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
# Quadratic Discriminant Analysis Model
def qda_cap(stock_name, upload = False):
    # Data split
    X_train, X_val1, X_val2, X_test, y_train, y_val1, y_val2, y_test = getProcessedData(getFilenames(fx_tic)[fx_dic[stock_name]])
    # For each market, we only use top 20 features selected by SFFS.
    X_train = X_train.iloc[:,feat_20[stock_name]]
    X_val1 = X_val1.iloc[:,feat_20[stock_name]]
    X_val2 = X_val2.iloc[:,feat_20[stock_name]]
    X_test = X_test.iloc[:,feat_20[stock_name]]

    # Check if the model should be loaded directly
    if upload == True:
        # For ensure that the model does not change when it is used again and is easy to reproduce.
        qda = load(path + '/model_weights/' + stock_name +'qda.h5')
        y_pred = qda.predict(X_val2)
        # Calculating the accuracy score
        test_score = qda.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
        
    # Otherwise, train the model from scratch
    elif upload == False:
        qda = QuadraticDiscriminantAnalysis()
        X_train1 = pd.concat([X_train, X_val1],axis=0)
        y_train1 = pd.concat([y_train, y_val1],axis=0)
        qda.fit(X_train1, y_train1)

        path1 = path + '/model_weights/' + stock_name +'qda.h5'
        dump(qda, path1)

        y_pred = qda.predict(X_val2)
        # Calculating the accuracy score
        test_score = qda.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
    else:
        print('argument upload should be True or False')

    return test_score, balanced_score1


# 5.Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Random Forest Model
def rf_cap(stock_name, upload = False):
    # Data split
    X_train, X_val1, X_val2, X_test, y_train, y_val1, y_val2, y_test = getProcessedData(getFilenames(fx_tic)[fx_dic[stock_name]])
    # For each market, we only use top 20 features selected by SFFS.
    X_train = X_train.iloc[:,feat_20[stock_name]]
    X_val1 = X_val1.iloc[:,feat_20[stock_name]]
    X_val2 = X_val2.iloc[:,feat_20[stock_name]]
    X_test = X_test.iloc[:,feat_20[stock_name]]

    # Check if the model should be loaded directly
    if upload == True:
        # For ensure that the model does not change when it is used again and is easy to reproduce.
        rf1 = load(path + '/model_weights/' + stock_name +'rf.h5')
        y_pred = rf1.predict(X_val2)
        # Calculating the accuracy score
        test_score = rf1.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
        
    # Otherwise, train the model from scratch
    elif upload == False:
        best_score = 0
        # Grid search to find the best hyperparameters for the Random Forest
        # 'bootstrap': Whether bootstrap samples are used when building trees
        for bootstrap in [True, False]:
            # 'max_features': The number of features to consider when looking for the best split
            for max_features in ['auto', 'sqrt']:
                # 'max_depth': Maximum depth of the tree
                for max_depth in [3,5,7,9]:
                    # 'min_samples_leaf': Minimum number of samples required at the leaf node
                    for min_samples_leaf in [1, 2, 4]:
                        # 'min_samples_split': Minimum number of samples required to split an internal node
                        for min_samples_split in [2, 5, 10]:
                                rf = RandomForestClassifier(bootstrap=bootstrap, max_features=max_features, max_depth=max_depth,
                                                          min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,
                                                          n_estimators = int(np.sqrt(len(X_train.columns))))
                                rf.fit(X_train, y_train)
                                score = rf.score(X_val1, y_val1)
                                # Update best parameters if current score is better
                                if score > best_score:
                                    best_score = score
                                    best_parameters = {'bootstrap': bootstrap, 'max_features': max_features, 'max_depth': max_depth,
                                                      'min_samples_leaf': min_samples_leaf, 'min_samples_split': min_samples_split,
                                                      }
        max_features1 = best_parameters['max_features']
        bootstrap1 = best_parameters['bootstrap']
        max_depth1 = best_parameters['max_depth']
        bootstrap1 = best_parameters['bootstrap']
        min_samples_leaf1 = best_parameters['min_samples_leaf']
        min_samples_split1 = best_parameters['min_samples_split']
        # Training the model with the best parameters
        rf1 = RandomForestClassifier(bootstrap=bootstrap1, max_features=max_features1, max_depth=max_depth1,
                                                          min_samples_leaf=min_samples_leaf1, min_samples_split=min_samples_split1,
                                                          n_estimators = int(np.sqrt(len(X_train.columns))))
        rf1.fit(X_train, y_train)
        # Saving the model
        path1 = path + '/model_weights/' + stock_name +'rf.h5'
        dump(rf1, path1)

        y_pred = rf1.predict(X_val2)
        # Calculating the accuracy score
        test_score = rf1.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
    else:
        print('argument upload should be True or False')

    return test_score, balanced_score1

# 6.XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
# XGBoost Model
def xgb_cap(stock_name, upload = False):
    # Data split
    X_train, X_val1, X_val2, X_test, y_train, y_val1, y_val2, y_test = getProcessedData(getFilenames(fx_tic)[fx_dic[stock_name]])
    # For each market, we only use top 20 features selected by SFFS.
    X_train = X_train.iloc[:,feat_20[stock_name]]
    X_val1 = X_val1.iloc[:,feat_20[stock_name]]
    X_val2 = X_val2.iloc[:,feat_20[stock_name]]
    X_test = X_test.iloc[:,feat_20[stock_name]]

    # Check if the model should be loaded directly
    if upload == True:
        # For ensure that the model does not change when it is used again and is easy to reproduce.
        xgb_model = load(path + '/model_weights/' + stock_name +'xgb.h5')
        y_pred = xgb_model.predict(X_val2)
        # Calculating the accuracy score
        test_score = xgb_model.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
    
    # Otherwise, train the model from scratch
    elif upload == False:
        best_score = 0
        # Grid search to find the best hyperparameters for the XGBoost
        # 'max_depth': Maximum depth of a tree
        for max_depth in [3,5,7,9]:
            # 'min_child_weight': Minimum sum of instance weight needed in a child
            for min_child_weight in [1, 2, 4]:
                # 'gamma': Regularization parameter
                for gamma in [0, 0.1, 0.2]:
                    # 'subsample': Proportion of training data to grow trees and prevent overfitting
                    for subsample in [0.6, 0.8, 1]:
                        # 'colsample_bytree': Subsample ratio of columns when constructing each tree
                        for colsample_bytree in [0.6, 0.8, 1]:
                            # 'learning_rate': Shrinks the feature weights to make the boosting process more conservative
                            for learning_rate in [0.001, 0.01, 0.1]:
                                xgb_model = xgb.XGBClassifier(learning_rate=learning_rate, max_depth=max_depth,
                                                              min_child_weight=min_child_weight, gamma=gamma,
                                                              subsample=subsample, colsample_bytree=colsample_bytree,
                                                              use_label_encoder=False, eval_metric='logloss',
                                                              n_estimators = int(np.sqrt(len(X_train.columns))))
                                xgb_model.fit(X_train, y_train)
                                score = xgb_model.score(X_val1, y_val1)
                                # Update best parameters if current score is better
                                if score > best_score:
                                    best_score = score
                                    best_parameters = {'max_depth': max_depth, 'min_child_weight': min_child_weight,
                                                      'gamma': gamma, 'subsample': subsample, 'colsample_bytree': colsample_bytree,
                                                      'learning_rate': learning_rate}
        # Training the model with the best parameters
        xgb_model = xgb.XGBClassifier(learning_rate=best_parameters['learning_rate'], max_depth=best_parameters['max_depth'],
                                      min_child_weight=best_parameters['min_child_weight'], gamma=best_parameters['gamma'],
                                      subsample=best_parameters['subsample'], colsample_bytree=best_parameters['colsample_bytree'],
                                      use_label_encoder=False, eval_metric='logloss',
                                      n_estimators = int(np.sqrt(len(X_train.columns))))
        xgb_model.fit(X_train, y_train)
        # Saving the model
        path1 = path + '/model_weights/' + stock_name +'xgb.h5'
        dump(xgb_model, path1)

        y_pred = xgb_model.predict(X_val2)
        # Calculating the accuracy score
        test_score = xgb_model.score(X_val2,y_val2)
        # Calculating the balanced accuracy score
        balanced_score1 = balanced_accuracy_score(y_val2, y_pred)
    else:
        print('argument upload should be True or False')

    return test_score, balanced_score1



# Result

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Lists to store accuracy scores and balanced accuracy scores for each model and market
Stnm = fx_tic
svmnon=[]
lr=[]
lda = []
qda = []
rf=[]
xgboost=[]
svmnon_balance=[]
lr_balance=[]
lda_balance = []
qda_balance = []
rf_balance = []
xgboost_balance = []

# Loop through each market
for name in fx_tic:
    a1,b1 = svmnon_cap(name)
    a2,b2 = lr_cap(name)
    a3,b3 = lda_cap(name)
    a4,b4 = qda_cap(name)
    a5,b5 = rf_cap(name)
    a6,b6 = xgb_cap(name)

    # Append scores to the lists
    svmnon.append(a1)
    lr.append(a2)
    lda.append(a3)
    qda.append(a4)
    rf.append(a5)
    xgboost.append(a6)

    svmnon_balance.append(b1)
    lr_balance.append(b2)
    lda_balance.append(b3)
    qda_balance.append(b4)
    rf_balance.append(b5)
    xgboost_balance.append(b6)


# Create a dataframe to compile all the accuracy scores
acc_df=pd.DataFrame(zip(Stnm,svmnon, svmnon_balance, lr, lr_balance, lda, lda_balance, qda, qda_balance, rf, rf_balance, xgboost, xgboost_balance),columns=['Stock','SVM Gaussian Kernel','SVM Gaussian Kernel balance',
                                                      'Logistic Regression','Logistic Regression balance','LDA','LDA balance','QDA','QDA balance','Random Forest','Random Forest balance','XGBoost','XGBoost balance'])

In [ ]:
acc_df

,Stock,SVM Gaussian Kernel,SVM Gaussian Kernel balance,Logistic Regression,Logistic Regression balance,LDA,LDA balance,QDA,QDA balance,Random Forest,Random Forest balance,XGBoost,XGBoost balance
0,USDEUR,0.516667,0.500000,0.516667,0.500000,0.535897,0.529991,0.492308,0.494395,0.525641,0.522889,0.510256,0.500727
1,USDJPY,0.485897,0.505691,0.478205,0.500000,0.501282,0.505092,0.482051,0.472331,0.483333,0.490580,0.510256,0.518171
2,USDGBP,0.524359,0.501700,0.523077,0.500000,0.517949,0.513599,0.484615,0.492528,0.512821,0.496007,0.488462,0.474146
3,USDCHF,0.506410,0.516176,0.487179,0.499485,0.497436,0.505821,0.542308,0.535737,0.512821,0.519838,0.489744,0.497466
4,USDNZD,0.535897,0.504808,0.533333,0.500000,0.511538,0.503434,0.506410,0.510474,0.469231,0.475446,0.524359,0.524210
5,USDCAD,0.491026,0.490906,0.494872,0.500000,0.488462,0.489604,0.496154,0.497035,0.505128,0.505234,0.528205,0.528734
6,USDSEK,0.496154,0.496154,0.496154,0.500000,0.494872,0.495651,0.512821,0.512003,0.484615,0.485078,0.507692,0.508906
7,USDDKK,0.491026,0.511982,0.460256,0.494931,0.484615,0.496378,0.502564,0.487404,0.507692,0.509768,0.534615,0.537478
8,USDNOK,0.526923,0.527684,0.496154,0.496745,0.521795,0.521108,0.514103,0.513710,0.474359,0.474305,0.528205,0.529081
9,EURJPY,0.515385,0.504264,0.515385,0.500000,0.469231,0.460831,0.484615,0.489418,0.476923,0.468767,0.510256,0.504580


In [ ]:
acc_df.to_csv(path + '/ml_acc_df_new_data.csv', index=False)